# SAMOS Spectroscopy - Complete Reduction with Wavelength Calibration

**Comprehensive spectroscopic reduction pipeline**

## Purpose

This notebook provides complete spectroscopic reduction:

**Part A: Trace Fitting & Extraction**
1. Fit trace curvature
2. Create spatial masks
3. Rectify 2D spectra
4. Background subtraction
5. Extract 1D spectra (boxcar + optimal)

**Part B: Wavelength Calibration**
6. Load arc lamp spectrum
7. Identify emission lines
8. Fit wavelength solution
9. Assess calibration quality

**Part C: Batch Processing**
10. Apply to all slits
11. Create calibrated FITS files
12. Generate QA plots

## Prerequisites

- Run `01_initial_inspection.ipynb` first
- Optionally run `02_visual_qa.ipynb` for QA

---

## 1. Import Packages

In [ ]:
# Standard libraries
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from astropy.io import fits
from astropy import units as u
from specutils import Spectrum1D
import warnings
warnings.filterwarnings('ignore')

# SAMOS modules
from samos.spectroscopy import extraction, pypeit_wrapper
from samos.utils import display, io

# Jupyter display
%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 6)

print("✓ All packages loaded")

## 2. Configuration

In [ ]:
# Target information
target_name = "ABELL3120"
target_mode = "SAMI_manual_Mask_T00_Low_Red"

# Working directory
analysis_top_directory = "/Users/nestrada/Documents/SAMOS/SAMOS_REDUCED"
working_directory = os.path.join(analysis_top_directory, target_name, target_mode)

os.chdir(working_directory)
print(f"Working directory: {working_directory}")

# Load QA configuration if available
import yaml
qa_config_file = 'qa_config.yaml'

if Path(qa_config_file).exists():
    with open(qa_config_file) as f:
        qa_config = yaml.safe_load(f)
    reference_slit = qa_config.get('reference_slit', 10)
    good_slits = qa_config.get('good_slits', None)
    print(f"✓ Loaded QA configuration")
    print(f"  Reference slit: {reference_slit}")
    if good_slits:
        print(f"  Good slits: {len(good_slits)}")
else:
    print("ℹ️  No QA configuration found, using defaults")
    reference_slit = 10
    good_slits = None

# Find spec files
spec_files = sorted(Path('.').glob('spec_*.fits'))
if good_slits is not None:
    # Filter to good slits only
    spec_files = [f for f in spec_files 
                  if int(f.stem.split('_')[1]) in good_slits]

n_slits = len(spec_files)
print(f"\nFound {n_slits} slit files to process")

---

# Part A: Trace Fitting & Extraction

---

## 3. Demonstrate Extraction Pipeline on Reference Slit

In [ ]:
# Load reference slit
ref_file = f'spec_{reference_slit:03d}.fits'
print(f"Loading reference slit: {ref_file}\n")

with fits.open(ref_file) as hdul:
    data = hdul['DATA'].data
    flat = hdul['FLAT'].data
    lines = hdul['LINES'].data
    header = hdul[0].header
    slit_edges = (header['Y0'], header['Y1'], header['Y2'], header['Y3'])

print(f"Slit {reference_slit} loaded:")
print(f"  Data shape: {data.shape}")
print(f"  Slit edges: {slit_edges}")

### Step 3a: Fit Trace from Flat Field

In [ ]:
# Find trace edges
print("Fitting trace curvature...")
x_pix, y_ctr = extraction.find_trace_edges(flat, x_range=(1500, 2000))

# Fit polynomial
trace_poly = extraction.fit_trace(x_pix, y_ctr, method='polynomial', degree=3)

print(f"✓ Trace fit complete")
print(f"  Found {len(x_pix)} edge points")
print(f"  Polynomial coefficients: {trace_poly.coefficients}")

# Visualize fit
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 5))

# Flat with trace overlay
vmin, vmax = display.get_percentile_limits(flat, 1, 99)
ax1.imshow(flat, origin='lower', cmap='gray', vmin=vmin, vmax=vmax, aspect='auto')
x_plot = np.arange(flat.shape[1])
ax1.plot(x_plot, trace_poly(x_plot), 'r-', linewidth=2, label='Trace fit')
ax1.scatter(x_pix, y_ctr, c='cyan', s=1, alpha=0.5, label='Detected edges')
ax1.set_xlabel('X (dispersion)')
ax1.set_ylabel('Y (spatial)')
ax1.set_title(f'Slit {reference_slit} - Trace Fitting')
ax1.legend()

# Residuals
residuals = y_ctr - trace_poly(x_pix)
ax2.scatter(x_pix, residuals, s=5, alpha=0.5)
ax2.axhline(0, color='r', linestyle='--')
ax2.set_xlabel('X pixel')
ax2.set_ylabel('Residual (pixels)')
ax2.set_title(f'Trace Fit Residuals (RMS = {np.std(residuals):.3f} pixels)')
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print(f"✓ Trace fitting complete")

### Step 3b: Create Spatial Masks

In [ ]:
# Create masks separating slit from sky
mask_slit, mask_sky = extraction.create_spatial_mask(
    data.shape, trace_poly, slit_edges, margin=12
)

# Visualize masks
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 4))

# Original data
vmin, vmax = display.get_percentile_limits(data, 1, 99)
ax1.imshow(data, origin='lower', cmap='gray', vmin=vmin, vmax=vmax, aspect='auto')
ax1.set_title('Science Data')
ax1.set_xlabel('X (dispersion)')
ax1.set_ylabel('Y (spatial)')

# Slit mask
ax2.imshow(mask_slit, origin='lower', cmap='gray', aspect='auto')
ax2.set_title('Slit Mask (illuminated region)')
ax2.set_xlabel('X (dispersion)')
ax2.set_ylabel('Y (spatial)')

# Sky mask
ax3.imshow(mask_sky, origin='lower', cmap='gray', aspect='auto')
ax3.set_title('Sky Mask (background region)')
ax3.set_xlabel('X (dispersion)')
ax3.set_ylabel('Y (spatial)')

plt.tight_layout()
plt.show()

print(f"✓ Masks created")
print(f"  Slit pixels: {np.sum(mask_slit)}")
print(f"  Sky pixels: {np.sum(mask_sky)}")

### Step 3c: Rectify Spectrum

In [ ]:
# Rectify to straighten the trace
data_rect = extraction.rectify_spectrum(data, trace_poly, center_y=20)
mask_slit_rect = extraction.rectify_spectrum(mask_slit.astype(float), trace_poly, center_y=20).astype(bool)
mask_sky_rect = extraction.rectify_spectrum(mask_sky.astype(float), trace_poly, center_y=20).astype(bool)

# Visualize rectification
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(18, 8))

# Original (curved)
vmin, vmax = display.get_percentile_limits(data, 1, 99)
ax1.imshow(data, origin='lower', cmap='gray', vmin=vmin, vmax=vmax, aspect='auto')
ax1.plot(x_plot, trace_poly(x_plot), 'r-', linewidth=1, label='Trace')
ax1.set_title('Original Spectrum (curved trace)')
ax1.set_xlabel('X (dispersion)')
ax1.set_ylabel('Y (spatial)')
ax1.legend()

# Rectified (straight)
vmin, vmax = display.get_percentile_limits(data_rect, 1, 99)
ax2.imshow(data_rect, origin='lower', cmap='gray', vmin=vmin, vmax=vmax, aspect='auto')
ax2.axhline(20, color='r', linewidth=1, label='Trace (now straight)')
ax2.set_title('Rectified Spectrum (straight trace)')
ax2.set_xlabel('X (dispersion)')
ax2.set_ylabel('Y (spatial)')
ax2.legend()

plt.tight_layout()
plt.show()

print("✓ Spectrum rectified")

### Step 3d: Background Subtraction

In [ ]:
# Subtract sky/background
data_clean = extraction.subtract_background(data_rect, mask_sky_rect, method='median')

# Visualize background subtraction
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 4))

vmin, vmax = display.get_percentile_limits(data_rect, 1, 99)
ax1.imshow(data_rect, origin='lower', cmap='gray', vmin=vmin, vmax=vmax, aspect='auto')
ax1.set_title('Before Background Subtraction')

vmin, vmax = display.get_percentile_limits(data_clean, 1, 99)
ax2.imshow(data_clean, origin='lower', cmap='gray', vmin=vmin, vmax=vmax, aspect='auto')
ax2.set_title('After Background Subtraction')

# Comparison profile
x_mid = data.shape[1] // 2
ax3.plot(data_rect[:, x_mid], label='Before', alpha=0.7)
ax3.plot(data_clean[:, x_mid], label='After', alpha=0.7)
ax3.set_xlabel('Y pixel')
ax3.set_ylabel('Counts')
ax3.set_title(f'Spatial Profile (x={x_mid})')
ax3.legend()
ax3.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("✓ Background subtracted")

### Step 3e: Extract 1D Spectra

In [ ]:
# Extract using both methods
spec1d_boxcar = extraction.extract_1d_boxcar(data_clean, mask_slit=mask_slit_rect)
spec1d_optimal, variance_1d = extraction.extract_1d_optimal(
    data_clean, mask_slit=mask_slit_rect
)

# Compare extraction methods
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(18, 8))

# Spectral comparison
ax1.plot(spec1d_boxcar, label='Boxcar', alpha=0.7)
ax1.plot(spec1d_optimal, label='Optimal', alpha=0.7)
ax1.set_xlabel('Pixel')
ax1.set_ylabel('Flux (ADU)')
ax1.set_title('Extracted 1D Spectra - Comparison')
ax1.legend()
ax1.grid(alpha=0.3)

# S/N for optimal extraction
snr = spec1d_optimal / np.sqrt(variance_1d)
ax2.plot(snr, color='green')
ax2.set_xlabel('Pixel')
ax2.set_ylabel('S/N')
ax2.set_title(f'Signal-to-Noise Ratio (Optimal Extraction)')
ax2.axhline(snr[snr>0].mean(), color='red', linestyle='--', 
           label=f'Mean S/N = {snr[snr>0].mean():.1f}')
ax2.legend()
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print(f"✓ 1D extraction complete")
print(f"  Mean S/N: {snr[snr>0].mean():.1f}")

---

# Part B: Wavelength Calibration

---

## 4. Prepare Arc Lamp Spectrum

In [ ]:
# Rectify arc lamp using the same trace
arc_rect = extraction.rectify_spectrum(lines, trace_poly, center_y=20)

# Extract 1D arc spectrum
arc_1d = np.sum(arc_rect * mask_slit_rect, axis=0)

# Display arc
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(18, 8))

# 2D arc (rectified)
vmin, vmax = display.get_percentile_limits(arc_rect, 1, 99)
ax1.imshow(arc_rect, origin='lower', cmap='gray', vmin=vmin, vmax=vmax, aspect='auto')
ax1.set_title('Rectified Arc Lamp (2D)')
ax1.set_xlabel('X (dispersion)')
ax1.set_ylabel('Y (spatial)')

# 1D arc spectrum
ax2.plot(arc_1d, color='orange')
ax2.set_xlabel('Pixel')
ax2.set_ylabel('Counts')
ax2.set_title('Arc Lamp Spectrum (1D)')
ax2.set_ylim([0, np.percentile(arc_1d, 99)])
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("✓ Arc lamp prepared")

## 5. Wavelength Calibration

Identify arc lines and fit wavelength solution.

In [ ]:
# Run wavelength calibration
print("Running wavelength calibration...\n")

solution = pypeit_wrapper.run_wavelength_calibration_simple(
    arc_1d,
    line_list='HgArNe',
    wv_range=(5500, 9000)
)

print(f"\nWavelength Solution:")
print(f"  Polynomial degree: {len(solution['coeffs']) - 1}")
print(f"  Matched lines: {solution['n_lines']}")
print(f"  RMS residual: {solution['rms']:.3f} Å")
print(f"  Coefficients: {solution['coeffs']}")

## 6. Wavelength Calibration Quality Assessment

In [ ]:
# Visualize wavelength solution
fig, axes = plt.subplots(2, 2, figsize=(18, 10))

# Arc with identified lines
ax1 = axes[0, 0]
ax1.plot(arc_1d, color='gray', alpha=0.5)
ax1.scatter(solution['matched_pixels'], arc_1d[solution['matched_pixels'].astype(int)],
           color='red', s=50, zorder=5, label=f"{solution['n_lines']} lines matched")
ax1.set_xlabel('Pixel')
ax1.set_ylabel('Counts')
ax1.set_title('Arc Spectrum with Identified Lines')
ax1.set_ylim([0, np.percentile(arc_1d, 99)])
ax1.legend()
ax1.grid(alpha=0.3)

# Wavelength solution
ax2 = axes[0, 1]
ax2.scatter(solution['matched_pixels'], solution['matched_wavelengths'], 
           s=50, alpha=0.7, label='Matched lines')
x_plot = np.linspace(0, len(arc_1d), 1000)
wl_plot = np.polyval(solution['coeffs'], x_plot)
ax2.plot(x_plot, wl_plot, 'r-', linewidth=2, label='Polynomial fit')
ax2.set_xlabel('Pixel')
ax2.set_ylabel('Wavelength (Å)')
ax2.set_title('Wavelength Solution')
ax2.legend()
ax2.grid(alpha=0.3)

# Residuals
ax3 = axes[1, 0]
ax3.scatter(solution['matched_pixels'], solution['residuals'], s=50)
ax3.axhline(0, color='r', linestyle='--')
ax3.axhline(solution['rms'], color='orange', linestyle=':', 
           label=f'RMS = {solution["rms"]:.3f} Å')
ax3.axhline(-solution['rms'], color='orange', linestyle=':')
ax3.set_xlabel('Pixel')
ax3.set_ylabel('Residual (Å)')
ax3.set_title('Wavelength Fit Residuals')
ax3.legend()
ax3.grid(alpha=0.3)

# Wavelength-calibrated arc
ax4 = axes[1, 1]
wavelengths = np.polyval(solution['coeffs'], np.arange(len(arc_1d)))
ax4.plot(wavelengths, arc_1d, color='orange')
for i, wl in enumerate(solution['matched_wavelengths'][:10]):
    ax4.axvline(wl, color='red', alpha=0.3, linewidth=0.5)
    if i < 5:
        ax4.text(wl, np.max(arc_1d)*0.9, f'{wl:.0f}', 
                rotation=90, fontsize=8, ha='right')
ax4.set_xlabel('Wavelength (Å)')
ax4.set_ylabel('Counts')
ax4.set_title('Wavelength-Calibrated Arc Spectrum')
ax4.set_xlim([wavelengths[0], wavelengths[-1]])
ax4.grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Save wavelength solution
wl_solution_file = f'wavelength_solution_{reference_slit:03d}.txt'
np.savetxt(wl_solution_file, solution['coeffs'])
print(f"\n✓ Wavelength solution saved to: {wl_solution_file}")

## 7. Wavelength-Calibrated Spectrum (Reference Slit)

In [ ]:
# Apply wavelength solution to extracted spectrum
pixels = np.arange(len(spec1d_optimal))
wavelengths_full = pypeit_wrapper.apply_wavelength_solution(pixels, solution)

# Trim to valid range
valid = (wavelengths_full >= 5800) & (wavelengths_full <= 9500)
wavelengths_trim = wavelengths_full[valid]
spec1d_trim = spec1d_optimal[valid]

# Plot wavelength-calibrated spectrum
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(18, 8))

# Full spectrum
ax1.plot(wavelengths_trim, spec1d_trim)
ax1.set_xlabel('Wavelength (Å)')
ax1.set_ylabel('Flux (ADU)')
ax1.set_title(f'Slit {reference_slit} - Wavelength-Calibrated 1D Spectrum')
ax1.grid(alpha=0.3)

# Zoomed region
zoom_range = (wavelengths_trim > 7000) & (wavelengths_trim < 7500)
ax2.plot(wavelengths_trim[zoom_range], spec1d_trim[zoom_range])
ax2.set_xlabel('Wavelength (Å)')
ax2.set_ylabel('Flux (ADU)')
ax2.set_title('Zoomed Region (7000-7500 Å)')
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print(f"✓ Wavelength range: {wavelengths_trim[0]:.1f} - {wavelengths_trim[-1]:.1f} Å")
print(f"  Dispersion: {(wavelengths_trim[-1]-wavelengths_trim[0])/len(wavelengths_trim):.3f} Å/pixel")

---

# Part C: Batch Processing

---

## 8. Apply to All Slits

Use the wavelength solution from the reference slit and apply to all slits.

In [ ]:
# Create output directory
output_dir = Path('./calibrated')
output_dir.mkdir(exist_ok=True)

print(f"Processing {n_slits} slits...\n")
print("=" * 80)

# Process each slit
results = {}

for i, spec_file in enumerate(spec_files):
    slit_num = int(spec_file.stem.split('_')[1])
    
    print(f"[{i+1}/{n_slits}] Slit {slit_num}...", end=' ')
    
    try:
        # Extract spectrum
        extract_results = extraction.full_extraction_pipeline(
            str(spec_file),
            config={
                'trace_fit': {'method': 'polynomial', 'degree': 3},
                'extraction': {'width': 10},
                'background': {'method': 'median'}
            }
        )
        
        # Create wavelength-calibrated FITS
        out_1d = output_dir / f'spec1d_{slit_num:03d}.fits'
        out_2d = output_dir / f'spec2d_{slit_num:03d}.fits'
        
        pypeit_wrapper.create_wavelength_fits(
            str(spec_file), solution, str(out_1d), str(out_2d), extract_results
        )
        
        results[slit_num] = {
            'success': True,
            'spec1d': str(out_1d),
            'spec2d': str(out_2d)
        }
        
        print("✓")
        
    except Exception as e:
        print(f"⚠️ Error: {e}")
        results[slit_num] = {'success': False, 'error': str(e)}

print("=" * 80)

n_success = sum(1 for r in results.values() if r['success'])
print(f"\n✓ Batch processing complete")
print(f"  Successful: {n_success}/{n_slits}")
print(f"  Failed: {n_slits - n_success}/{n_slits}")
print(f"\nOutput directory: {output_dir}")

## 9. Verify Output Files

In [ ]:
# List created files
spec1d_files = sorted(output_dir.glob('spec1d_*.fits'))
spec2d_files = sorted(output_dir.glob('spec2d_*.fits'))

print(f"Output files:")
print(f"  1D spectra: {len(spec1d_files)} files")
print(f"  2D spectra: {len(spec2d_files)} files")

# Display a few spectra
n_display = min(5, len(spec1d_files))

fig, axes = plt.subplots(n_display, 1, figsize=(18, 3*n_display))
if n_display == 1:
    axes = [axes]

for i, spec_file in enumerate(spec1d_files[:n_display]):
    spec1d = Spectrum1D.read(spec_file)
    
    axes[i].plot(spec1d.spectral_axis, spec1d.flux)
    axes[i].set_xlabel('Wavelength (Å)')
    axes[i].set_ylabel('Flux (ADU)')
    axes[i].set_title(f'{spec_file.name}')
    axes[i].grid(alpha=0.3)
    
    # Show wavelength range
    wl_min = spec1d.spectral_axis.min().value
    wl_max = spec1d.spectral_axis.max().value
    axes[i].text(0.02, 0.95, f'{wl_min:.0f} - {wl_max:.0f} Å',
                transform=axes[i].transAxes, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

print(f"\n✓ Output verified")

## 10. Summary Statistics

In [ ]:
import pandas as pd

# Collect summary statistics
summary_data = []

for spec_file in spec1d_files:
    slit_num = int(spec_file.stem.split('_')[1])
    spec1d = Spectrum1D.read(spec_file)
    
    wl_min = spec1d.spectral_axis.min().value
    wl_max = spec1d.spectral_axis.max().value
    n_pixels = len(spec1d.spectral_axis)
    dispersion = (wl_max - wl_min) / n_pixels
    median_flux = np.median(spec1d.flux.value)
    
    summary_data.append({
        'Slit': slit_num,
        'λ_min (Å)': f"{wl_min:.1f}",
        'λ_max (Å)': f"{wl_max:.1f}",
        'N_pixels': n_pixels,
        'Dispersion (Å/pix)': f"{dispersion:.3f}",
        'Median Flux': f"{median_flux:.1f}"
    })

summary_df = pd.DataFrame(summary_data)

print("\n" + "=" * 100)
print("WAVELENGTH-CALIBRATED SPECTRA SUMMARY")
print("=" * 100)
print(summary_df.to_string(index=False))
print("=" * 100)

print(f"\nReduction Summary:")
print(f"  Target: {target_name}")
print(f"  Mode: {target_mode}")
print(f"  Reference slit: {reference_slit}")
print(f"  Wavelength solution RMS: {solution['rms']:.3f} Å")
print(f"  Processed slits: {len(spec1d_files)}")
print(f"  Output directory: {output_dir}")

---

## Summary

This notebook has completed:

**Part A: Trace Fitting & Extraction**
1. ✓ Fit trace curvature from flat field
2. ✓ Created spatial masks (slit vs sky)
3. ✓ Rectified 2D spectra
4. ✓ Subtracted background/sky
5. ✓ Extracted 1D spectra (boxcar + optimal)

**Part B: Wavelength Calibration**
6. ✓ Prepared arc lamp spectrum
7. ✓ Identified emission lines (HgArNe)
8. ✓ Fit wavelength solution (polynomial)
9. ✓ Assessed calibration quality

**Part C: Batch Processing**
10. ✓ Applied to all slits
11. ✓ Created wavelength-calibrated FITS files
12. ✓ Generated summary statistics

**Next Steps:**

Proceed to `04_visualization.ipynb` for:
- Interactive visualization with jdaviz
- Specviz for 1D spectral analysis
- Mosviz for multi-object spectroscopy

**Files Created:**
- `calibrated/spec1d_*.fits` - Wavelength-calibrated 1D spectra
- `calibrated/spec2d_*.fits` - Wavelength-calibrated 2D spectra
- `wavelength_solution_*.txt` - Wavelength solution coefficients

---

**Complete SAMOS spectroscopic reduction finished!** 🎉